<a href="https://colab.research.google.com/github/Manjuphoenix/ML_team_buzz/blob/michelle/v1_apptom_knowledge_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from collections import Counter

from PIL import Image

import torch
import torchvision
from torch import utils
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
import torchvision.models as models
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler, ConcatDataset



from sklearn.model_selection import train_test_split
from tqdm import tqdm



torch.cuda.empty_cache()

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

!pip install kaggle

from google.colab import files 
files.upload()

#Make a directory named kaggle and copy the kaggle.json file there; and change its permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#imagenet_data = torchvision.datasets.ImageNet('https://kaggle.com/saroz014/plant-diseases')
#data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=4, shuffle=True, num_workers=args.nThreads)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


In [4]:
!kaggle datasets download -d manjuphoenix/tomatodataset
!kaggle datasets download -d manjuphoenix/appledataset

100% 921M/923M [00:06<00:00, 156MB/s]
100% 923M/923M [00:06<00:00, 153MB/s]
100% 332M/333M [00:03<00:00, 118MB/s]
100% 333M/333M [00:03<00:00, 97.7MB/s]


In [5]:
from zipfile import ZipFile
file_name = 'tomatodataset.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  #zip.printdir()
  print(zip.extractall())
  #zip.close()
  print('Done')

None
Done


In [6]:
from zipfile import ZipFile
file_name = 'appledataset.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  #zip.printdir()
  print(zip.extractall())
  #zip.close()
  print('Done')

None
Done


In [ ]:
# from zipfile import ZipFile
# file_name = 'cherrydataset.zip' #the file is your dataset exact name
# with ZipFile(file_name, 'r') as zip:
#   #zip.printdir()
#   print(zip.extractall())
#   #zip.close()
#   print('Done')

None
Done


In [ ]:
# from zipfile import ZipFile
# file_name = 'corndataset.zip' #the file is your dataset exact name
# with ZipFile(file_name, 'r') as zip:
#   #zip.printdir()
#   print(zip.extractall())
#   #zip.close()
#   print('Done')

None
Done


In [7]:
!mkdir /content/dataset
!sudo cp -rf /content/Apple/* /content/dataset
!sudo cp -rf /content/Tomato/* /content/dataset
# !sudo cp -rf /content/cherry/* /content/dataset
# !sudo cp -rf /content/corn/* /content/dataset

In [51]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomGrayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
])


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Uncomment the below line based on where you train the model----------------------------------------
# !mkdir /kaggle/working/Tomato

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# !cp -r /kaggle/input/plant-diseases/dataset_itr2/dataset_itr2/test/Tomato* /kaggle/working/Tomato
# !cp -r /kaggle/input/plant-diseases/dataset_itr2/dataset_itr2/train/Tomato* /kaggle/working/Tomato

# !rm -rf /kaggle/working/Tomato/Tomato___Leaf_Mold
# !rm -rf /kaggle/working/Tomato/Tomato___Tomato_mosaic_virus 

# Uncomment this for training on kaggle
# data = datasets.ImageFolder('/kaggle/working/Tomato', transform=transform)----------------------------


# This is for training on Local Machine
data = datasets.ImageFolder('/content/dataset', transform=transform)


# Split into train/test sets:
train_len = int(len(data)*0.7)
train_set, test_set = random_split(data, [train_len, len(data) - train_len])

# Extract classes:
train_classes = [train_set.dataset.targets[i] for i in train_set.indices]
# Calculate support:
class_count = Counter(train_classes)
# Calculate class weights:
class_weights = torch.DoubleTensor([len(train_classes)/c for c in pd.Series(class_count).sort_index().values]) 
# Sampler needs the respective class weight supplied for each image in the dataset:
sample_weights = [class_weights[train_set.dataset.targets[i]] for i in train_set.indices]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=int(len(train_set)*2), replacement=True)

batch_size=32

# Create torch dataloaders:

dataloaders = DataLoader(data, batch_size=batch_size, sampler=sampler, num_workers=min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8]))
print("The total number of images is:", len(dataloaders))

train_loader = DataLoader(train_set, batch_size=batch_size, sampler=sampler, num_workers=min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8]))
print("The number of images in a training set is:", len(train_loader)*batch_size)

val_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8]))
print("The number of images in a test set is:", len(val_loader)*batch_size)
print(dataloaders.dataset)

print(data.classes)
# x, y = next(iter(dataloaders[0]))

cuda:0
The total number of images is: 3993
The number of images in a training set is: 127776
The number of images in a test set is: 27392
Dataset ImageFolder
    Number of datapoints: 91256
    Root location: /content/dataset
    StandardTransform
Transform: Compose(
               ToTensor()
               RandomGrayscale(p=0.1)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
           )
['Apple___healthy', 'Apple_scab', 'Black_rot', 'Cedar_apple_rust', 'Tomato_Target_spot', 'Tomato__Late_blight', 'Tomato__Septoria_leaf_spot', 'Tomato__Spider_mites', 'Tomato___Bacterial_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato__early_blight', 'Tomato__healthy']


In [67]:
import torch
from torchvision.transforms import transforms
from PIL import Image
from pathlib import Path


# # To get the weights from the .pth file!
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.cnn = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu = nn.LeakyReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=2)
        self.relu2 = nn.LeakyReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32*65*65, 1000)
        self.fc2 = nn.Linear(1000, 14)

    def forward(self, x):
        out = self.cnn(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [68]:
model = NeuralNetwork()
checkpoint = torch.load(Path('/content/gdrive/MyDrive/v2_apptom.pk1'))
model.load_state_dict(checkpoint)

model = model.to(device)
print(model)

NeuralNetwork(
  (cnn): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): LeakyReLU(negative_slope=0.01)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (relu2): LeakyReLU(negative_slope=0.01)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=135200, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=12, bias=True)
)


In [69]:
model.eval()

NeuralNetwork(
  (cnn): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): LeakyReLU(negative_slope=0.01)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (relu2): LeakyReLU(negative_slope=0.01)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=135200, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=12, bias=True)
)

In [70]:
def evaluate(model, dataset, max_ex=0):
    acc = 0
    N = len(dataset) * batch_size
    for i, (features, labels) in enumerate(dataset):
        features = features.to(device)
        labels = labels.to(device)
        scores = model(features)
        pred = torch.argmax(scores, dim=1)
        acc += torch.sum(torch.eq(pred, labels)).item()
    # print(i)
    return (acc * 100 / ((i+1) * batch_size) )


In [71]:
train_acc = evaluate(model, train_loader)
print("\nTrain accuracy: %.2f%%" % train_acc)
val_acc = evaluate(model,val_loader)
print("Validation accuracy: %.2f%%" % val_acc)
test_acc = evaluate(model, val_loader)
print("Test accuracy: %.2f%%" % test_acc)


Train accuracy: 82.34%
Validation accuracy: 82.53%
Test accuracy: 82.24%


In [72]:
### Define our custom loss function
optim_fun = nn.Softmax(dim=1)
loss_fn = nn.CrossEntropyLoss()

def my_loss(scores, targets, T=5):
    soft_pred = optim_fun(scores / T)
    soft_targets = optim_fun(targets / T)
    loss = loss_fn(soft_pred, soft_targets)
    return loss

In [73]:
class StudentNeuralNetwork(nn.Module):
    def __init__(self):
        super(StudentNeuralNetwork, self).__init__()
        self.cnn = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32*128*128, 12)

    def forward(self, x):
        out = self.cnn(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [74]:
student_network = StudentNeuralNetwork()
student_network.eval()

StudentNeuralNetwork(
  (cnn): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=524288, out_features=12, bias=True)
)

In [75]:
def evaluate(model, dataset, max_ex=0):
    acc = 0
    N = len(dataset) * batch_size
    for i, (features, labels) in enumerate(dataset):
        features = features.to(device)
        labels = labels.to(device)
        scores = model(features)
        pred = torch.argmax(scores, dim=1)
        acc += torch.sum(torch.eq(pred, labels)).item()
    # print(i)
    return (acc * 100 / ((i+1) * batch_size) )

In [76]:
import matplotlib.pyplot as plt

def plot_loss(loss, it, it_per_epoch, smooth_loss=[], base_name='', title=''):
    fig = plt.figure(figsize=(8, 4), dpi=100)
    plt.plot(loss)
    plt.plot(smooth_loss)
    epochs = [i * int(it_per_epoch) for i in range(int(it / it_per_epoch) + 1)]
    plt.plot(epochs, [loss[i] for i in epochs], linestyle='', marker='o')
    # if len(epochs) > 1: print(smooth_loss[epochs[-2]] -  smooth_loss[epochs[-1]] )
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Iteration')
    # plt.ylim([0, 3])
    if base_name != '':
        fig.savefig(base_name + '.png')
        # pickle.dump([loss, smooth_loss, it], open(base_name + '-' + str(it) + '.p', 'wb'))
        #print(it)
    else:
        plt.show()
    plt.close("all")

def plot_acc(train_acc, val_acc, it, it_per_epoch, base_name='', title=''):
    fig = plt.figure(figsize=(8, 4), dpi=100)
    if it !=0:
        inter = it//(len(train_acc) -1)
        x_axis = [i * inter for i in range(len(train_acc))]
    else:
        x_axis = [0]
    plt.plot(x_axis, train_acc, label="Train")
    plt.plot(x_axis, val_acc, label="Validation")
    plt.legend()
    # epochs = [i * int(it_per_epoch) for i in range(int(it / it_per_epoch) + 1)]
    # plt.plot(epochs, [smooth_loss[i] for i in epochs], linestyle='', marker='o')
    # if len(epochs) > 1: print(smooth_loss[epochs[-2]] -  smooth_loss[epochs[-1]] )
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Iteration')
    plt.ylim([90, 100])
    if base_name != '':
        fig.savefig(base_name + '.png')
        # pickle.dump([loss, smooth_loss, it], open(base_name + '-' + str(it) + '.p', 'wb'))
        #print(it)
    else:
        plt.show()
    plt.close("all")

In [77]:
### Define our custom loss function
softmax_op = nn.Softmax(dim=1)
mseloss_fn = nn.CrossEntropyLoss()

def my_loss(scores, targets, T=5):
    soft_pred = softmax_op(scores / T)
    soft_targets = softmax_op(targets / T)
    loss = mseloss_fn(soft_pred, soft_targets)
    return loss

In [ ]:
small_model = StudentNeuralNetwork().to(device)
big_model = NeuralNetwork().to(device)
from tqdm import tqdm
%matplotlib inline

# Hyperparameters
lr = 5e-3
epochs = 5
temp = 5
it_per_epoch = len(train_loader)
it = 0


# Create optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.0003, weight_decay=1e-5, momentum=0.6)
val_acc = []
train_acc = []
train_loss = [0]  # loss at iteration 0
for epoch in range(epochs):
    for features, labels in tqdm(train_loader):
        features = features.to(device)
        labels = labels.to(device)
        scores = small_model(features)
        targets = big_model(features)
        loss = my_loss(scores, targets, T = temp)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Book-keeping
        train_acc.append(evaluate(small_model, train_loader, max_ex=100))
        val_acc.append(evaluate(small_model, val_loader))
#perform last book-keeping
train_acc.append(evaluate(small_model, train_loader, max_ex=100))
val_acc.append(evaluate(small_model, val_loader))
plot_acc(train_acc, val_acc, it, it_per_epoch)

  0%|          | 7/3993 [21:44<206:13:42, 186.26s/it]